In [ ]:
# | default_exp core

# Core

> Core pieces needed to use during code generation.

In [ ]:
# | hide
import jupyter_black

In [ ]:
# | hide
jupyter_black.load()

In [ ]:
# | export
from typing import Any
from copy import deepcopy
from textwrap import indent
from collections import ChainMap
from typing import Callable, Optional
from jinja2 import Environment, BaseLoader, Template, StrictUndefined
from frontmatter.default_handlers import YAMLHandler
from frontmatter.util import u
from textwrap import dedent

In [ ]:
# | hide
import nbdev
import nbdev.showdoc as showdoc
from fastcore.test import *
from fastcore.imports import *
from jinja2 import UndefinedError

## Data

We want a code generator, but before, we need a class to hold data while doing code generation. For this reason, the first class we are developing is creatively named `Data`

We'll see later that our nodes/elements will be nested, creating a parent-children. Instead of backing that in, we'll create that with a mixin, because we'll need it for other purposes later.

In [ ]:
# | export
class WithChildrenMixin:
    """
    Adds `parent`/`children` functionality to a class.
    """

    def __init__(self):
        self.parent = None
        self.children = []

    def __len__(self):
        return len(self.children)

    def __contains__(self, element):
        return element in self.children

    def append(self, child: "Data"):
        """
        Add a child element to the children list and set its parent to self.
        """
        self.children.append(child)
        child.set_parent(self)
        return child

    def set_parent(self, parent: "Data"):
        """
        Set the parent element of self.
        """
        self.parent = parent

    def __iter__(self):
        def iter_data(obj, level=0):
            """Simply yields parent and then children"""
            yield obj, level
            for child in obj.children:
                yield from iter_data(child, level=level + 1)

        return iter_data(self)

So now, we can make the class to hold the data

In [ ]:
# | export
class Data(WithChildrenMixin):
    """
    Data holder used during code generation. Logic is kept as separate functions.
    """

    def __init__(
        self,
        name: str,  # Name of this element
        attrs: dict[str, Any] = None,  # Attributes for this element
    ):
        """
        Initialize Data object.

        """

        self.name = name

        if attrs is None:
            attrs = {}
        self._attrs = attrs

        super().__init__()

    @property
    def attrs(self):
        """
        Get the attributes for this element, merged with parent's attributes, if available.

        """
        if self.parent:
            return ChainMap(self._attrs, self.parent.attrs)
        return ChainMap(self._attrs)

    def clone(self):
        """
        Create a deep copy of this Data object.

        """
        return deepcopy(self)

    def __eq__(self, a):
        """
        Compare this Data object with another object for equality.

        """
        print("==")
        same_name = self.name == a.name
        same_attrs = self.attrs == a.attrs
        same_children = self.children == a.children
        return same_name and same_attrs and same_children

    def __str__(self):
        """
        Get the string representation of this Data object.

        """
        is_self_closing = not self.children

        if self.children:
            children = map(str, self.children)
            children = "\n".join(children)
            children = children.strip()
            children = f"\n{children}\n"
            children = indent(children, "    ")

        if self.attrs:
            if is_self_closing:
                return f"<{self.name} {dict(self.attrs)} />"
            else:
                return f"<{self.name} {dict(self.attrs)}>{children}</{self.name}>"

        if is_self_closing:
            return f"<{self.name} />"
        else:
            return f"<{self.name}>{children}</{self.name}>"

    __repr__ = __str__

In [ ]:
# | hide
showdoc.show_doc(Data)

---

[source](https://github.com/mintyPT/sal/blob/main/sal/core.py#L47){target="_blank" style="float:right; font-size:smaller"}

### Data

>      Data (name:str, attrs:dict[str,typing.Any]=None)

Data holder used during code generation. Logic is kept as separate functions.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str |  | Name of this element |
| attrs | dict | None | Attributes for this element |

### Basic operations

Now that we have a class to hold our data, and only for now, we declare structures to use for code generation, manually. Later, we'll create some data-loaders.

In [ ]:
james = Data("person", {"name": "james"})

print(james)

# ---
test_eq(james.name, "person")
test_eq(james.attrs, {"name": "james"})

<person {'name': 'james'} />


We can add children (note: a child's attributes will also include those of his parent)

In [ ]:
james = Data("person", {"name": "james"})
olive = james.append(Data("person", {"name": "olive"}))
silva = james.append(Data("person", {"name": "silva"}))

andrew = olive.append(Data("person", {"name": "andrew"}))

john = james.append(Data("person", {"name": "john"}))
jane = olive.append(Data("person", {"name": "jane"}))
noname = olive.append(Data("person"))

# ---
test_eq(james.children[0].attrs["name"], "olive")

and a child will know its parent

In [ ]:
print(olive.parent.attrs["name"])

# ---
assert james == olive.parent
# test_eq(james, olive.parent)

james
==


To check the number of children, simply use `len`

In [ ]:
len(james)

# ---
assert len(james) == 3

You can compare elements but they are tested based on their attributes and children

In [ ]:
b = Data("b", {"age": 22})
c = Data("b", {"age": 22})
d = Data("d")
b.append(d)
c.append(d)

# ---
assert b == c
# test_eq(b, c)
assert Data("b", {"name": "santos"}) == Data("b", {"name": "santos"})
# test_eq(Data("b", {"name": "santos"}), Data("b", {"name": "santos"}))
assert Data("b") != Data("c")
# test_ne(Data("b"), Data("c"))
assert Data("b", {"name": "silva"}) != Data("b", {"name": "santos"})
# test_ne(Data("b", {"name": "silva"}), Data("b", {"name": "santos"}))

==
==
==
==


You can test if an element is a child of another

In [ ]:
test_eq(olive in james, True)

### Cloning

You can duplicate any `Data` instance

In [ ]:
james.clone()

<person {'name': 'james'}>
    <person {'name': 'olive'}>
        <person {'name': 'andrew'} />
        <person {'name': 'jane'} />
        <person {'name': 'olive'} />
    </person>
    <person {'name': 'silva'} />
    <person {'name': 'john'} />
</person>

In [ ]:
# | hide

root = Data("root")
root.append(Data("child"))

boot = root
assert root == boot

root.attrs["extra"] = "please"
assert root == boot


shoe = root.clone()
shoe.attrs["extra"] = "please2"
assert root != shoe

==
==
==
==


### Logic

#### Basic iteration of all the elements

If you just need to iterate through all the elements, a simple loop will suffice

In [ ]:
for person, level in james:
    print("   " * level, person.name + "::" + person.attrs["name"])

 person::james
    person::olive
       person::andrew
       person::jane
       person::olive
    person::silva
    person::john


#### Map

To do a `map-like` sequencial processing on data and transform it to anything else, use `map_data` which will return instances of `MappedData`

In [ ]:
# | export
class MappedData(WithChildrenMixin):
    """Data structure used to return results from the `map_data` function"""

    def __init__(self, value):
        self.value = value
        super().__init__()

In [ ]:
# | export
def map_data(obj: Data, process: Callable, level=0) -> MappedData:
    """Maps over a `Data` inst returning `MappedData` instances"""
    child_results = [map_data(c, process, level=level + 1) for c in obj.children]
    value = process(obj, level)
    data = MappedData(value)
    for c in child_results:
        data.append(c)
    return data

Here's an example showing how to process all the `Data` instances in a tree...

In [ ]:
def fun(data: Data, level: int = 0):
    name = data.attrs["name"]
    level_str = " " * (level + 1)
    return f"{level_str}-I'm {name}-"


result = map_data(james.clone(), fun)

...and print the result

In [ ]:
for obj, level in result:
    print(obj.value)

 -I'm james-
  -I'm olive-
   -I'm andrew-
   -I'm jane-
   -I'm olive-
  -I'm silva-
  -I'm john-


In [ ]:
# | hide
test_eq(result.children[0].value, "  -I'm olive-")

## Jinja2

In [ ]:
# | export
# | hide
def _get_env():
    return Environment(loader=BaseLoader(), undefined=StrictUndefined)

We need a rendering function capable of renderering a `template`, with `filters` and `context variables`...

In [ ]:
# | export
# | hide
def render(
    template: str,  # template in string form
    filters: Optional[dict] = None,  # jinja filters
    **kwargs: Any,
) -> str:
    if not filters:
        filters = {}

    env = _get_env()
    env.filters.update(filters)

    jinja: Template = env.from_string(template)
    result: str = jinja.render(**kwargs)

    return result

In [ ]:
showdoc.show_doc(render)

---

[source](https://github.com/mintyPT/sal/blob/main/sal/core.py#L157){target="_blank" style="float:right; font-size:smaller"}

### render

>      render (template:str, filters:Optional[dict]=None, **kwargs:Any)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| template | str |  | template in string form |
| filters | Optional | None | jinja filters |
| kwargs | Any |  |  |
| **Returns** | **str** |  |  |

In [ ]:
# | hide
filters = {"upper": lambda n: n.upper()}
template = 'this is a template is this is my name "{{ name | upper }}" in upper case'
kwargs = dict(name="mauro")

result = render(template, filters=filters, **kwargs)
expected = 'this is a template is this is my name "MAURO" in upper case'
test_eq(result, expected)

try:
    render("{{ name }}")
except UndefinedError:
    pass

## Frontmatter

Later on, we will use `frontmatter` to make our code generator more powerful. Let's build some helpers

In [ ]:
# | hide
# | export



class FrontMatter:
    def __init__(self, handler=None):
        if handler is None:
            handler = YAMLHandler()
        self.handler = handler

    def split(self, raw_content, *, encoding="utf-8"):
        raw_content = u(raw_content, encoding).strip()

        try:
            fm, content = self.handler.split(raw_content)
        except ValueError:
            return None, raw_content

        return fm, content

    def parse(self, raw_frontmatter, *, metadata=None):
        if metadata is None:
            metadata = {}

        try:
            raw_frontmatter = self.handler.load(raw_frontmatter)
        except Exception as e:
            msg = dedent(
                f"""
            ===
            There is an error with the following yaml (front matter)
            
            ```
            {raw_frontmatter}
            ```

            ===

            """
            )

            print(msg)
            raise e

        if isinstance(raw_frontmatter, dict):
            metadata.update(raw_frontmatter)

        return metadata

    def get_content(self, template):
        frontmatter, content = self.split(template)
        return content.strip()

    def get_raw_frontmatter(self, template):
        resp = self.split(template)
        frontmatter, content = resp
        if frontmatter:
            return frontmatter.strip()

## Argument parsing

In [ ]:
# | export
import json

In [ ]:
# | export


def parse_arg(arg):
    try:
        v = json.loads(arg)
    except json.JSONDecodeError:
        v = arg
    return v

In [ ]:
# TODO test this
# def eval_arg(arg, locals_):
#    return eval(arg, None, locals_)

In [ ]:
# | hide
for raw, expected in [
    # number
    ["1", 1],
    ["1.1", 1.1],
    # string
    ["santos", "santos"],
    # json
    ['{"name": "santos"}', {"name": "santos"}],
    ["[1, 2, 3]", [1, 2, 3]],
    ['["1", "2"]', ["1", "2"]],
    # bool
    ["true", True],
    ["false", False],
]:
    v = parse_arg(raw)
    # print(v, type(v))
    test_eq(v, expected)

In [ ]:
# | export


def parse_attrs(attrs):
    for k, y in attrs.items():
        attrs[k] = parse_arg(y)
    return attrs

In [ ]:
# | hide
raw = {
    "name": "john",  # string
    "age": "1",  # int
    "money": "98123.45",  # float
    "happy": "true",  # bool
    "sad": "false",  # bool
    "colors": '["red", "blue"]',  # json
}
expected = {
    "name": "john",
    "age": 1,
    "money": 98123.45,
    "happy": True,
    "sad": False,
    "colors": ["red", "blue"],
}
parsed = parse_attrs(raw)

test_eq(parsed, expected)

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()